In [1]:
import numpy as np
import copy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from sumo_interface import start_sim

In [2]:
def random_dur(default_dur, sigma=1):
	default_dur = sum(default_dur,[])
	result =  np.clip(np.random.normal(default_dur,sigma*np.ones(len(default_dur)),len(default_dur)),0.5,60)
	#print("generated: ",result)
	return result

In [3]:
def evaluation(file_path,id_TLs,dur_TLs_):
	dur_TLs = copy.deepcopy(dur_TLs_)
	#print("eval ind: ",dur_TLs)
	edit_net(file_path,id_TLs,dur_TLs)
	#--> to sumo api

	start_sim(file_path+"/osm.sumocfg")
	import xml.etree.ElementTree as ET
	from numpy import mean
	tree = ET.ElementTree(file = "/home/lin/workspace/GA/GA_Traffic_Optimization/result.xml")
	#tree = ET.ElementTree(file = "/home/zhenyuli/workspace/GA_Traffic_Optimization/result.xml")
	trip_infos = tree.getroot()
	timeLoss = mean([float(trip.attrib['timeLoss']) for trip in trip_infos.findall("tripinfo")])

	print("##############################################################")
	print("##############################################################")
	print("avg time loss: ", timeLoss)
	print("##############################################################")
	print("##############################################################")
	return timeLoss,

In [4]:
def checkBounds(min, max):
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] > max:
                        child[i] = max
                    elif child[i] < min:
                        child[i] = min
            return offspring
        return wrapper
    return decorator

In [5]:
def ga(file_path,default_dur,id_TLs):
	#parameters
	INIT_SIZE = 100
	MAX_ITER = 30
	SIGMA = 1
	P_CROSSOVER = 0.5
	P_MUTATION = 0.1
	MIN = 0.5
	MAX = 60

	#DEAP
	creator.create("FitnessMin",base.Fitness,weights=(-1.0,)) 
	creator.create("Individual",list,fitness=creator.FitnessMin) #individuals are returned in list

	toolbox = base.Toolbox()
	toolbox.register("attr_item", random_dur, default_dur, SIGMA)
	toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_item)
	toolbox.register("population", tools.initRepeat, list, toolbox.individual)

	toolbox.register("evaluate", evaluation, file_path,id_TLs)
	toolbox.register("edit_net", edit_net,file_path,id_TLs,dur_TLs)
	toolbox.register("start_sim",start_sim,file_path)
	toolbox.register("mate", tools.cxOnePoint)
	#toolbox.register("mate", crossover)23.789012557077626
	toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=SIGMA, indpb=0.1)
	toolbox.decorate("mutate", checkBounds(MIN, MAX))# Bounds are still needed to be set.

	toolbox.register("select", tools.selTournament, tournsize=5)
	#toolbox.decorate("select", printsel())

	pop = toolbox.population(n=INIT_SIZE)

	hof = tools.HallOfFame(1) #pick the best one
	stats = tools.Statistics(lambda ind: ind.fitness.values)
	stats.register("avg", np.mean)
	stats.register("std", np.std)
	stats.register("min", np.min)
	stats.register("max", np.max)

	pop,log = algorithms.eaSimple(pop,toolbox,cxpb=P_CROSSOVER,mutpb=P_MUTATION,ngen=MAX_ITER,stats=stats,halloffame=hof,verbose=True)

	return hof,log

In [6]:
def get_default_duration(file_path):
	import xml.etree.ElementTree as ET

	tree = ET.ElementTree(file = file_path+"/osm.net.xml")
	net = tree.getroot()
	trafficLights = net.findall("tlLogic")

	id_TLs = []
	dur_TLs = []
	for tl in trafficLights:
		id_TLs.append(tl.attrib['id'])
		durations = [phase.attrib['duration'] for phase in tl]
		dur_TLs.append(durations)

	return id_TLs,dur_TLs

In [7]:
def edit_net(file_path,id_TLs,dur_TLs):
	import xml.etree.ElementTree as ET

	tree = ET.ElementTree(file = file_path+"/osm.net.xml")
	net = tree.getroot()
	trafficLights = net.findall("tlLogic")

	tl_idx = [[tl.attrib['id'] for tl in trafficLights].index(id) for id in id_TLs]

	for i in tl_idx:
		nb_phase = len(trafficLights[i])
		for j in range(nb_phase):
			trafficLights[i][j].attrib["duration"] = str(dur_TLs.pop(0))
		#for j in range(len(dur_TLs[i])):
		#	trafficLights[i][j].attrib["duration"] = dur_TLs[i][j]

	tree.write(file_path+"/osm.net.xml")

In [8]:
MAP_PATH = "/home/lin/workspace/GA/GA_Traffic_Optimization/wuhan"
#MAP_PATH = "/home/zhenyuli/Sumo/2020-02-08-00-46-38"
id_TLs, dur_TLs = get_default_duration(MAP_PATH)

best_solution,log = ga(MAP_PATH,dur_TLs,id_TLs)
print("best solution is: ",best_solution[0])
print("best time loss", best_solution[0].fitness.values)

 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  216.96445741953858
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  186.80426659071492
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  229.2836969524352
##############################################################
##############################################################
 Retrying in 1 seconds
################################################

##############################################################
##############################################################
avg time loss:  188.119313585873
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  131.95739390487043
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  195.06043862147536
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
#########

##############################################################
##############################################################
avg time loss:  277.39525776132155
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  152.53039589860438
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  173.93339504414698
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
#######

##############################################################
##############################################################
avg time loss:  164.08483053261182
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  215.5081600683566
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  153.0560922814013
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
#########

##############################################################
##############################################################
avg time loss:  187.45435203645684
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  138.82714041583594
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  157.69835944175446
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
#######

...

##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
#######

##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
#######

##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
29 	49    	110.077	16.9883	105.657	206.407
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
###########################

##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  169.9184534320706
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
########

##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  105.65734263742523
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
##############################################################
avg time loss:  206.5761207633153
##############################################################
##############################################################
 Retrying in 1 seconds
##############################################################
########

In [11]:
gen = log.select("gen")
fit_mins = log.select("min")
import plotly.graph_objects as go
import numpy as np

x = np.arange(10)

fig = go.Figure(data=go.Scatter(x=gen, y=fit_mins, mode='lines+markers'))
fig.update_layout(title='The performance of GA')
fig.show()

In [10]:
print(log)

gen	nevals	avg    	std    	min    	max    
0  	100   	184.781	45.6871	120.228	417.532
1  	49    	164.674	39.5483	116.642	357.407
2  	49    	158.082	38.3499	116.642	338.514
3  	58    	152.682	44.4506	105.963	391.74 
4  	53    	145.825	35.634 	105.963	265.108
5  	53    	137.171	34.9195	105.963	267.272
6  	62    	138.52 	40.1925	105.963	263.341
7  	54    	123.311	35.3549	105.963	299.841
8  	47    	108.613	12.8641	105.963	209.418
9  	48    	112.052	19.2385	105.963	215.829
10 	52    	110.119	16.01  	105.963	194.082
11 	39    	109.971	16.2118	105.657	193.671
12 	60    	107.226	8.02254	105.657	180.729
13 	50    	112.853	19.5283	105.657	194.124
14 	64    	116.863	28.8894	105.657	277.691
15 	52    	113.31 	20.6917	105.657	201.126
16 	41    	109.776	16.1404	105.657	213.247
17 	60    	111.717	23.3344	105.657	241.541
18 	60    	112.593	23.3529	105.657	219.274
19 	46    	111.847	25.1089	105.657	276.535
20 	64    	112.963	22.9021	105.657	218.305
21 	61    	111.816	22.4207	105.657	266.98 
22 	51    	